In [ ]:
import plotly.graph_objects as go
from plotly.colors import sequential
from itertools import combinations
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
import plotly.graph_objects as go

# from scipy.spatial import ConvexHull

In [ ]:
def recursive_discriminant(expr, vars):
    '''
    doc string :)
    '''
    vars_list = list(vars)
    def recursive_discriminant_helper(expr, vars):
        # if there are no variables, return the expression
        if vars == []:
            return expr
        
        for var in vars_list:
            if expr.has(var):
                vars_list.remove(var)
                return sp.discriminant(recursive_discriminant(expr, vars_list), var)
            
    return recursive_discriminant_helper(expr, vars_list)

def dual_hull(expr):
    '''
    doc string :)
    '''
    # get the variables  and dual variables
    vars = expr.free_symbols
    dual_vars = [sp.symbols(f"{var}_d") for var in vars]
    G = sp.symbols('G') # this is dual of the energy

    # compute the plane difference
    plane_diff = expr
    for var, dual_var in zip(vars, dual_vars):
        plane_diff -= dual_var * var
    plane_diff -= G

    # compute the dual hull
    dual_hull = recursive_discriminant(plane_diff, vars)

    # reduce the coefficients
    coeffs = set(term.as_coeff_mul()[0] for term in dual_hull.args)
    gcd_value = abs(sp.gcd(*coeffs))
    dual_hull = dual_hull / gcd_value

    return dual_hull

#### Here we are going to hard-code everything just to get it working

In [ ]:
# real space variables
x, y, z = sp.symbols('x y z')

# dual space variables
x_d, y_d, G = sp.symbols("x_d y_d G")

# define your funcitons here in explicit form
f1 = x**2 + y**2
f2 = (x-1)**2 + (y+1)**2 + 1
f3 = (x+1)**2 + (y+2)**2 + 1

display(f1, f2, f3)

In [ ]:
hp = x_d*x + y_d*y + G

f1_d = dual_hull(f1)
f2_d = dual_hull(f2)
f3_d = dual_hull(f3)

f1_c = hp.subs(G, sp.solve(f1_d, G)[0])
f2_c = hp.subs(G, sp.solve(f2_d, G)[0])
f3_c = hp.subs(G, sp.solve(f3_d, G)[0])

display(f1_c, f2_c, f3_c)

In [ ]:
f12_d = sp.resultant(f1_d, f2_d, G)
f13_d = sp.resultant(f1_d, f3_d, G)
f23_d = sp.resultant(f2_d, f3_d, G)

f12_c = f1_c.subs(y_d, sp.solve(f12_d, y_d)[0])
f13_c = f1_c.subs(y_d, sp.solve(f13_d, y_d)[0])
f23_c = f2_c.subs(y_d, sp.solve(f23_d, y_d)[0])

display(f12_c, f13_c, f23_c)

In [ ]:
f123_d = sp.resultant(f12_d, f13_d, y_d)

f123_c = f12_c.subs(x_d, sp.solve(f123_d, x_d)[0])

display(f123_c)

In [ ]:
# find the intersection of the planes with the paraboloids
f = f1
eq1 = sp.Eq(sp.diff(f, x), sp.diff(f123_c, x))
eq2 = sp.Eq(sp.diff(f, y), sp.diff(f123_c, y))

point_f1_f123_x = list(round(float(i.evalf()),2) for i in sp.solve((eq1, eq2), (x, y)).values())

f = f2
eq1 = sp.Eq(sp.diff(f, x), sp.diff(f123_c, x))
eq2 = sp.Eq(sp.diff(f, y), sp.diff(f123_c, y))

point_f2_f123_x = list(round(float(i.evalf()),2) for i in sp.solve((eq1, eq2), (x, y)).values())

f = f3
eq1 = sp.Eq(sp.diff(f, x), sp.diff(f123_c, x))
eq2 = sp.Eq(sp.diff(f, y), sp.diff(f123_c, y))

point_f3_f123_x = list(round(float(i.evalf()),2) for i in sp.solve((eq1, eq2), (x, y)).values())

points = [point_f1_f123_x, point_f2_f123_x, point_f3_f123_x]
print(points)

In [ ]:
# find the boundry lines
# f1
f = f1
fc = f12_c
eq1 = sp.solve(sp.Eq(sp.diff(f, x), sp.diff(fc, x)), x_d)
eq2 = sp.Eq(sp.diff(f, y), sp.diff(fc, y)).subs(x_d, eq1[0])

line_f1_f12c = sp.solve(eq2, y)[0]
line_f1_f12c

f = f1
fc = f13_c
eq1 = sp.solve(sp.Eq(sp.diff(f, x), sp.diff(fc, x)), x_d)
eq2 = sp.Eq(sp.diff(f, y), sp.diff(fc, y)).subs(x_d, eq1[0])

line_f1_f13_c = sp.solve(eq2, y)[0]
line_f1_f13_c

 # f2
f = f2
fc = f12_c
eq1 = sp.solve(sp.Eq(sp.diff(f, x), sp.diff(fc, x)), x_d)
eq2 = sp.Eq(sp.diff(f, y), sp.diff(fc, y)).subs(x_d, eq1[0])

line_f2_f12c = sp.solve(eq2, y)[0]
line_f2_f12c

f = f2
fc = f23_c
eq1 = sp.solve(sp.Eq(sp.diff(f, x), sp.diff(fc, x)), x_d)
eq2 = sp.Eq(sp.diff(f, y), sp.diff(fc, y)).subs(x_d, eq1[0])

line_f2_f23c = sp.solve(eq2, y)[0]
line_f2_f23c

# f3
f = f3
fc = f13_c
eq1 = sp.solve(sp.Eq(sp.diff(f, x), sp.diff(fc, x)), x_d)
eq2 = sp.Eq(sp.diff(f, y), sp.diff(fc, y)).subs(x_d, eq1[0])

line_f3_f13c = sp.solve(eq2, y)[0]
line_f3_f13c

f = f3
fc = f23_c
eq1 = sp.solve(sp.Eq(sp.diff(f, x), sp.diff(fc, x)), x_d)
eq2 = sp.Eq(sp.diff(f, y), sp.diff(fc, y)).subs(x_d, eq1[0])

line_f3_f23c = sp.solve(eq2, y)[0]
line_f3_f23c

lines = [line_f1_f12c, line_f1_f13_c, line_f2_f12c, line_f2_f23c, line_f3_f13c, line_f3_f23c]
print(lines)


In [ ]:
# import numpy as np
# import plotly.graph_objects as go

# # Define the grids for the XY plane
# x_range = np.linspace(-3, 3, 500)
# y_range = np.linspace(-3, 3, 500)
# x_range, y_range = np.meshgrid(x_range, y_range)

# # define your funcitons here in explicit form
# f1 = x**2 + y**2
# f2 = (x-1)**2 + (y+1)**2 + 1
# f3 = (x+1)**2 + (y+2)**2 + 1

# # Compute z values for each paraboloid at each (x, y) point
# z1 = sp.lambdify([x, y], f1, 'numpy')(x_range, y_range)  # Paraboloid 1
# z2 = sp.lambdify([x, y], f2, 'numpy')(x_range, y_range)  # Paraboloid 2
# z3 = sp.lambdify([x, y], f3, 'numpy')(x_range, y_range)  # Paraboloid 3

# # Create surface plots for each paraboloid
# surface1 = go.Surface(x=x_range, y=y_range, z=z1, colorscale='Viridis', opacity=0.9, name='Paraboloid 1')
# surface2 = go.Surface(x=x_range, y=y_range, z=z2, colorscale='Cividis', opacity=0.9, name='Paraboloid 2')
# surface3 = go.Surface(x=x_range, y=y_range, z=z3, colorscale='Plasma', opacity=0.9, name='Paraboloid 3')

# # Define the boundary lines in the XY-plane
# # Replace these with your actual boundary line data

# x_range = np.linspace(-3, 3, 500)
# y_range = np.linspace(-3, 3, 500)
# line_z = np.zeros_like(x_range)

# line1 = sp.lambdify(x, lines[0], 'numpy')(x_range)
# line3 = sp.lambdify(x, lines[1], 'numpy')(x_range)
# line2 = sp.lambdify(x, lines[2], 'numpy')(x_range)
# line4 = sp.lambdify(x, lines[3], 'numpy')(x_range)
# line5 = sp.lambdify(x, lines[4], 'numpy')(x_range)
# line6 = sp.lambdify(x, lines[5], 'numpy')(x_range)

# # Create line plots for the boundary lines
# line1 = go.Scatter3d(x=x_range, y=line1, z=line_z, mode='lines', line=dict(color='black'))
# line2 = go.Scatter3d(x=x_range, y=line2, z=line_z, mode='lines', line=dict(color='black'))
# line3 = go.Scatter3d(x=x_range, y=line3, z=line_z, mode='lines', line=dict(color='black'))
# line4 = go.Scatter3d(x=x_range, y=line4, z=line_z, mode='lines', line=dict(color='black'))
# line5 = go.Scatter3d(x=x_range, y=line5, z=line_z, mode='lines', line=dict(color='black'))
# line6 = go.Scatter3d(x=x_range, y=line6, z=line_z, mode='lines', line=dict(color='black'))

# # Define the triangle points in the XY-plane

# # Define the triangle points in the XY-plane
# triangle_x = [p[0] for p in points] + [points[0][0]]  # Close the triangle
# triangle_y = [p[1] for p in points] + [points[0][1]]
# triangle_z = [0] * len(triangle_x)

# # Create a line plot for the triangle
# triangle = go.Scatter3d(x=triangle_x, y=triangle_y, z=triangle_z, mode='lines', line=dict(color='red'), name='Triangle')

# # Combine all plots into a single figure
# fig = go.Figure(data=[surface1, surface2, surface3, line1, line2, line3, line4, line5, line6, triangle])

# # Update layout for better visualization
# fig.update_layout(
#     width=900,
#     height=800, 
#     title='Paraboloids with Boundary Lines and Triangle',
#     scene=dict(
#         xaxis_title='X Axis',
#         yaxis_title='Y Axis',
#         zaxis_title='Z Axis',
#         zaxis=dict(range=[-1, 20]),
#         xaxis=dict(range=[-3, 3]),
#         yaxis=dict(range=[-3, 3])
#     )
# )

# # Display the plot
# fig.show()

In [ ]:
import numpy as np
import sympy as sp
from scipy.spatial import ConvexHull
import plotly.graph_objects as go
from collections import defaultdict

# Define symbols for sympy
x, y = sp.symbols('x y')

# Define the paraboloids
z1_expr = x**2 + y**2             # Paraboloid 1
z2_expr = (x - 2)**2 + (y - 2)**2 # Paraboloid 2
z3_expr = (x + 2)**2 + (y - 2)**2 # Paraboloid 3

# Lambdify the expressions for numerical computation
z1_func = sp.lambdify((x, y), z1_expr, 'numpy')
z2_func = sp.lambdify((x, y), z2_expr, 'numpy')
z3_func = sp.lambdify((x, y), z3_expr, 'numpy')

# Create a mesh grid over the xy-plane
grid_size = 100
x_vals = np.linspace(-5, 5, grid_size)
y_vals = np.linspace(-5, 5, grid_size)
X, Y = np.meshgrid(x_vals, y_vals)

# Compute z values for each paraboloid
Z1 = z1_func(X, Y)
Z2 = z2_func(X, Y)
Z3 = z3_func(X, Y)

# Flatten the arrays and combine the points
points = []
labels = []

# Paraboloid 1 points
for xi, yi, zi in zip(X.flatten(), Y.flatten(), Z1.flatten()):
    points.append([xi, yi, zi])
    labels.append(1)

# Paraboloid 2 points
for xi, yi, zi in zip(X.flatten(), Y.flatten(), Z2.flatten()):
    points.append([xi, yi, zi])
    labels.append(2)

# Paraboloid 3 points
for xi, yi, zi in zip(X.flatten(), Y.flatten(), Z3.flatten()):
    points.append([xi, yi, zi])
    labels.append(3)

points = np.array(points)
labels = np.array(labels)

# Compute the convex hull
hull = ConvexHull(points)

# Prepare face colors
face_colors = []
color_map = {'blue': 'rgba(0, 0, 255, 0.6)',    # Blue color with 60% opacity
             'red': 'rgba(255, 0, 0, 0.6)',     # Red color with 60% opacity
             'yellow': 'rgba(255, 255, 0, 0.6)'}# Yellow color with 60% opacity

for simplex in hull.simplices:
    face_labels = labels[simplex]
    unique_labels = np.unique(face_labels)
    if len(unique_labels) == 1:
        color = 'blue'  # All points from one paraboloid
    elif len(unique_labels) == 2:
        counts = defaultdict(int)
        for label in face_labels:
            counts[label] += 1
        if 2 in counts.values():
            color = 'red'   # Two points from one paraboloid, one from another
        else:
            color = 'yellow'  # Each point from a different paraboloid
    else:
        color = 'yellow'  # Each point from a different paraboloid
    face_colors.append(color_map[color])

# Prepare data for Plotly
x_pts = points[:, 0]
y_pts = points[:, 1]
z_pts = points[:, 2]

# Triangles are defined by indices i, j, k
i = hull.simplices[:, 0]
j = hull.simplices[:, 1]
k = hull.simplices[:, 2]

# Create the mesh
mesh = go.Mesh3d(
    x=x_pts,
    y=y_pts,
    z=z_pts,
    i=i,
    j=j,
    k=k,
    facecolor=face_colors,
    opacity=1.0,
    flatshading=True,
    showscale=False
)

# Create the figure
fig = go.Figure(data=[mesh])

# Update layout
fig.update_layout(
    scene=dict(
        xaxis=dict(range=[-5, 5], title='X-axis'),
        yaxis=dict(range=[-5, 5], title='Y-axis'),
        zaxis=dict(range=[0, np.max(z_pts)], title='Z-axis'),
    ),
    title='Convex Hull of Three Paraboloids'
)

# Show the figure
fig.show()
